In [1]:
flex_title = "Indonesia Fire Danger Rating System"
flex_subtitle = "Nusantara Earth Observation Network"
flex_custom_css = "custom.css"

In [2]:
import ee
from datetime import date, timedelta

# Mapping backend
import folium
import geemap.foliumap as geemap

# Interactive Python Widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

component_dict = {
    'Fire Weather Index' : {
        'prefix' : 'FWI',
        'min' : 0.0,
        'max' : 13.0,
    },
    'Initial Spread Index' : {
        'prefix' : 'ISI',
        'min' : 0.0,
        'max' : 5.0,
    },
    'Buildup Index' : {
        'prefix' : 'BUI',
        'min' : 0.0,
        'max' : 33.0,
    },
    'Drought Code' : {
        'prefix' : 'DC',
        'min' : 0.0,
        'max' : 350.0
    },
    'Duff Moisture Code' : {
        'prefix' : 'DMC',
        'min' : 0.0,
        'max' : 29.0
    },
    'Fine Fuel Moisture Code' : {
        'prefix' : 'FFMC',
        'min' : 0.0,
        'max' : 101.0
    }
}

province_dict = {
    'Sumatera Selatan' : {
        'prefix' : 'ID-SS',
    },
    'Jambi' : {
        'prefix' : 'ID-JA',
    },
    'Riau' : {
        'prefix' : 'ID-RI',
    }
}

palette = ['blue', 'green', 'yellow', 'red']

## Section

### Parameters

In [3]:
# Province selection
prov_label = widgets.Label(value='Province:')
prov_var = widgets.Dropdown(
    options=[
        'Sumatera Selatan',
        'Jambi',
        'Riau',
    ],
    value='Sumatera Selatan',
)

# Observation date selection
obs_label = widgets.Label(value='Date:')
obs_var = widgets.DatePicker(
    value=date.today() - timedelta(days=1)
)

# Component selection
comp_label = widgets.Label(value='Component:')
comp_var = widgets.Dropdown(
    options=[
        'Fine Fuel Moisture Code', 
        'Duff Moisture Code', 
        'Drought Code', 
        'Initial Spread Index', 
        'Buildup Index', 
        'Fire Weather Index'
    ],
    value='Fire Weather Index',
)

# Create and display the sidebar
inputs = widgets.VBox([prov_label, prov_var, obs_label, obs_var, comp_label, comp_var])
display(inputs)

### About

This application shows calibrated Canadian Fire Weather Index (FWI) calculated by using data from GSMaP (JAXA) and GFS (NOAA).

For more information, contact:
[Josef Matondang](mailto:admin@josefmtd.com)

## Column

### Fire Weather Index

In [11]:
def on_value_change(change):
    obs = obs_var.value
    comp = comp_var.value
    prov = prov_var.value

    with out:
        clear_output(wait=True)

        # Create folium map object
        fig = folium.Figure(height="100%")
        Map = geemap.Map(zoom=7, add_google_map=False).add_to(fig)

        # Obtain ee.Image from Google Cloud Storage
        prefix = province_dict[prov]['prefix']
        url = f'gs://fwi-bicubic-outputs/{prefix}_{obs.isoformat()}_FWI.tif'
        fwi_data = ee.Image.loadGeoTIFF(url)

        # Visualization parameters
        min_val = component_dict[comp]['min']
        max_val = component_dict[comp]['max']
        vis_params = {
            'min': min_val, 
            'max': max_val, 
            'palette': palette
        }

        # Add province parameters
        regencies = ee.FeatureCollection('FAO/GAUL/2015/level2') \
            .filter(ee.Filter.eq('ADM1_NAME', prov))

        province = ee.FeatureCollection('FAO/GAUL/2015/level1') \
            .filter(ee.Filter.eq('ADM1_NAME', prov))
        center = ee.Feature(province.first()).centroid().geometry().getInfo()['coordinates']

        # Add overlay layer
        layer = component_dict[comp]['prefix']
        Map.setCenter(lat=center[1], lon=center[0], zoom=7)
        Map.addLayer(fwi_data.select(layer), vis_params, comp)
        Map.addLayer(regencies, {}, prov)
        Map.add_colorbar(colors=palette, vmin=min_val, vmax=max_val)
        display(Map)

# Create output figure and callback functions to update the values
out = widgets.Output()
prov_var.observe(on_value_change, names="value")
obs_var.observe(on_value_change, names="value")
comp_var.observe(on_value_change, names="value")

In [12]:
# Shows the figure
on_value_change(None)
display(out)

Output()